In [4]:
import pandas as pd
import numpy as np
import os
import copy
import math
from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)

ModuleNotFoundError: No module named 'google.colab'

In [5]:
path = " Enter the folder path of Unzipped UCI Daphnet Dataset "
os.chdir(path)

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: ' Enter the folder path of Unzipped UCI Daphnet Dataset '

## To Label the Data with 3rd class (Pre-FOG)
Before the occurence of every FOG event, the previous w*f_s timesteps are labelled as a third class 'preFOG' which can be trained in order to predict FOG before it's onset.

In [6]:
def label_prefog(dataset,window_length = 1):
  dataset.drop(index = list(dataset[dataset['Action'] == 0].index),inplace=True)
  window_length = 64*window_length
    
  fog_index=[]
  for i in dataset.index: 
      if dataset.loc[i,'Action'] == 2:
        fog_index.append(i)
  fog_index



  start_indices=[]
  for i in fog_index:
    if (dataset.loc[i-1,'Action']!=dataset.loc[i,'Action']):
      start_indices.append(i)
 

  prefog=[]
  for start in start_indices:
    prefog_start = [x for x in range(start-window_length,start)]
    prefog.append(prefog_start)

  prefog = [item for sublist in prefog for item in sublist]

  for i in prefog:
       dataset.loc[i,'Action'] = 3
  dataset['Action'] = dataset['Action'] - 1
  return dataset

In [7]:
data_path = " Enter the folder path of Unzipped UCI Daphnet Dataset "

people = []
for person in os.listdir(data_path):
    if '.txt' in person: 
        people.append(person)
for window_length in range(1,5):
    for person in people: 
        name = person.split('R')[0]
        print (name)
        file = data_path+"\\"+person
        temp = pd.read_csv(file,delimiter= " ", header = None)
        print (person,' is read',end = '\t')
        if 2 in temp[max(temp.columns)].unique():
            print ('Adding {} to dataset'.format(person),end = '\t')
            temp.columns = ['time','A_F','A_V','A_L','L_F','L_V','L_L','T_F','T_V','T_L','Action']    
            temp = label_prefog(temp,window_length).reset_index(drop=True)
            temp['name'] = name
            print ('{} is labelled'.format(person))
            dataset = pd.concat([dataset,temp],axis = 0)

        print ('')
    dataset.reset_index(drop =True,inplace=True) 
    to_path = data_path + "\\raw_labelled"
    to_name = to_path +"\\win_"+str(window_length)+".csv"
     
    dataset.to_csv(to_name,index = False)


display(dataset.head())

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: ' Enter the folder path of Unzipped UCI Daphnet Dataset '

### To Extract Non-Overlapping windows of length w *f_s from the continously logged accelerometer data from the dataset.